<a href="https://colab.research.google.com/github/muk465/CS6910-Fundamentals-of-Deeplearning/blob/main/Assignment1/mse_loss_fmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
(trainx,trainy),(testx,testy)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
#onehot encoder
def onehot_encoder(Y):
  Y_encoded = np.zeros((10, Y.shape[0]))
  for train_point in range(Y.shape[0]):
      class_label = Y[train_point]
      Y_encoded[class_label,train_point] = 1.0
  return Y_encoded

In [ ]:
num_examples_train=int(0.9*trainx.shape[0])
num_examples_val=int(0.1*trainx.shape[0])
num_examples_test=testx.shape[0]


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(trainx, trainy, test_size=0.1,shuffle=True)

In [ ]:
X_train=x_train.reshape(x_train.shape[0],28*28).T
X_val=x_val.reshape(x_val.shape[0],28*28).T
X_test=testx.reshape(testx.shape[0],28*28).T
X_train=X_train/255
X_val=X_val/255
X_test=X_test/255


In [ ]:
Y_train=onehot_encoder(y_train)
Y_val=onehot_encoder(y_val)
Y_test=onehot_encoder(testy)


In [ ]:
!pip install wandb


     |████████████████████████████████| 1.7 MB 7.4 MB/s 
     |████████████████████████████████| 181 kB 64.9 MB/s 
     |████████████████████████████████| 144 kB 69.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=79525077a55c82ef4dcfd5a28f54fca143d996fcc3d6dea8e6809bdc22e21265
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import numpy as np
import wandb

In [ ]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-(z)))


def tanh(z):
    return np.tanh(z)


def sin(z):
    return np.sin(z)


def relu(z):
    return np.maximum(0, z) 


def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))


def grad_sigmoid(z):
    return  (1.0 / (1 + np.exp(-(z))))*(1 -  1.0 / (1 + np.exp(-(z))))

def grad_tanh(z):
    return 1 - np.tanh(z) ** 2

def grad_relu(z):
    if z>0:
      return 1
    else:
      return 0


In [ ]:
 #initializers
def xavier_initializer(op,ip):
  std = np.sqrt(2 / (ip + op))
  return np.random.normal(0, std, size=(op, ip))

def random_initializer(op,ip):
  return np.random.normal(0, 1, size=(op, ip))

In [ ]:
class neural_network2:

  def __init__(
      self, 
      num_hidden_layers,
      num_hidden_neurons, 
      X_train, 
      Y_train,  
      num_examples_train, 
      X_val, 
      Y_val, 
      num_examples_val,    
      optimizer,
      batch_size,
      lamda,
      learning_rate,
      max_epochs,
      activation,
      initializer,
      loss):
    #initalize neural nw layers
    self.layers = [784]+num_hidden_layers*[num_hidden_neurons]+[10]

    #initilialize num of examples
    self.num_examples_train = num_examples_train
    self.num_examples_val = num_examples_val
        
    #initlialize datset
    self.X_train=X_train
    self.X_val=X_val
    self.Y_train=Y_train
    self.Y_val=Y_val
   

    #loss fn
    self.loss_function = loss
    
    
    #setup initializer
    if initializer=="random":
      self.initializer = random_initializer
    elif initializer=="xavier":
      self.initializer=xavier_initializer
    else:
      raise Exception('this is wrong initializer')
    
    #setup activations
    if activation=="sigmoid":
      self.activation=sigmoid
      self.grad_activation=grad_sigmoid
    elif activation=="tanh":
      self.activation=tanh
      self.grad_activation=grad_tanh
    elif activation=="relu":
      self.activation=relu
      self.grad_activation=grad_relu
    else:
      raise Exception('this is wrong activation')

    #optimizers
    if optimizer=="sgd":
      self.optimizer=self.sgd
    elif optimizer=="mgd":
      self.optimizer=self.mgd
    elif optimizer=="nag":
      self.optimizer=self.nag
    elif optimizer=="rmsprop":
      self.optimizer=self.rmsprop
    elif optimizer=="adam":
      self.optimizer=self.adam
    elif optimizer=="nadam":
      self.optimizer=self.nadam
    else:
      raise Exception("wrong optimizer")
         

    #hyperparameters
    self.max_epochs = max_epochs
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.lamda=lamda
    
    #initialize weights
    self.W, self.b = self.Neuralnet_init(self.layers)




  # loss fn
  def cross_entropy_loss(self, Y_true, Y_pred):
      loss=np.sum(-(Y_true*np.log(Y_pred)))
      return loss

  def L2_loss(self, lamda):
    return (lamda/2) * np.sum([np.linalg.norm((self.W[i + 1]**2))  for i in range(len(self.W))])

  def mse_loss(self, Y_true, Y_pred):
      loss = np.sum(0.5*(Y_true - Y_pred) ** 2)
      return loss

  #accuracy and predict
  def accuracy(self, Y_true, Y_pred, num_examples):
    acc = 0
    for i in range(num_examples):
      if (np.argmax(Y_true[:, i])==np.argmax(Y_pred[:, i])):
        acc+=1
    accuracy = acc / num_examples
    return accuracy

  def predict(self,X,num_examples):
    Y_pred = []        
    for i in range(num_examples):
      Y, H, A = self.forwardPropagate(X[:, i].reshape(784, 1),self.W,self.b)
      Y_pred.append(Y.reshape(10,))
    Y_pred = np.array(Y_pred).transpose()
    return Y_pred
 
  #neural network weights and biases initializer
  def Neuralnet_init(self,layer_neurons):
      num_layers=len(self.layers)
      W = {}
      b = {}
      for i in range(0, num_layers - 1):
          W[i + 1] = self.initializer(layer_neurons[i + 1], layer_neurons[i])
          b[i + 1] = np.zeros((layer_neurons[i + 1], 1))
      return W,b
  
  def forwardPropagate(self,X, W, b):
    num_layers=len(self.layers)
    H={}
    A = {}

    H[0] = X
    A[0] = X
    for k in range(0, num_layers - 2):
      A[k + 1] = np.dot(W[k + 1],H[k])+ b[k + 1]
      H[k + 1] = self.activation(A[k + 1])

    #op layer
    A[num_layers - 1] = np.dot(W[num_layers-1],H[num_layers - 2])+ b[num_layers - 1]
    y_pred = softmax(A[num_layers - 1])
    H[num_layers - 1] = y_pred
    return y_pred,H,A
  
  
  
  
  def backPropagate(self,Ypred,H,A,Y_train,lamda=0):
    
    num_layers=len(self.layers)
    grad_A={}
    grad_H={}

    grad_W={}
    grad_b={}
    

    # Gradient of op layer(AL).
    # for MSE assuming op activation as softmax
    grad_A[num_layers - 1] = -(Y_train - Ypred)*(Ypred)*(1-Ypred)
    #grad_A[num_layers - 1] = -(Y_train - Ypred)

    for k in range(num_layers - 2, -1, -1):
      grad_W[k + 1] = np.outer(grad_A[k + 1],H[k])+(lamda/2 )* self.W[k + 1]

        
      grad_b[k + 1]=grad_A[k + 1]
      
      if k == 0:
        grad_H[k] = np.dot(self.W[k + 1].T,grad_A[k + 1])
        grad_A[k] = grad_H[k]*A[k]
      else:
        grad_H[k] = np.dot(self.W[k + 1].T,grad_A[k + 1])
          
        grad_A[k] = grad_H[k]*self.grad_activation(A[k])


          
    return grad_W,grad_b
    


  def sgd(self,epochs,num_examples,batch_size,learning_rate, lamda):
    loss_train = []
    acc_train = []
    acc_val = []
    num_layers=len(self.layers)
    for epoch in range(epochs):
      loss_per_point=0
      
      for i in range(X_train.shape[1]):
        Y,H,A = self.forwardPropagate(self.X_train[:,i].reshape(784,1), self.W, self.b) 
        grad_w, grad_b = self.backPropagate(Y,H,A,self.Y_train[:,i].reshape(10,1))
      
        loss_per_point= loss_per_point+ self.cross_entropy_loss(Y_train[:, i].reshape(10, 1), Y)+self.L2_loss(lamda)

        for k in range(num_layers-1):
          self.W[k+1]=self.W[k+1] - learning_rate * grad_w[k+1]
          self.b[k+1]=self.b[k+1] - learning_rate * grad_b[k+1]
            
      
         #loss    
      loss_train.append(loss_per_point/X_train.shape[1])

      #accuracy 
      Y_pred_final = self.predict(self.X_train,self.num_examples_train)
      acc_train.append(self.accuracy(Y_train, Y_pred_final, num_examples))
      acc_val.append(self.accuracy(self.Y_val, self.predict(self.X_val, self.num_examples_val), self.num_examples_val))

      print("epoch:",epoch,"training loss:",loss_train[epoch],"training accuracy",acc_train[epoch],"validation accuracy",acc_val[epoch])

      wandb.log({'epoch':epoch,'training_loss':loss_train[epoch],'training_accuracy':acc_train[epoch], 'validation_accuracy':acc_val[epoch] })


    return Y_pred_final,acc_val



  def mgd(self,epochs,num_examples,batch_size,learning_rate, lamda):
    #gamma = min(1 - 2 ** (-1 - np.log((epoch / 250.0) + 1, 2)), gamma)
    gamma=0.9
    eta=learning_rate
    loss_train = []
    acc_train = []
    acc_val = []
    del_w={}
    del_b={}
    prev_update_w={}
    prev_update_b={}
    update_w={}
    update_b={}
    
    X_train = self.X_train[:, :num_examples]
    Y_train = self.Y_train[:, :num_examples] 
    num_layers=len(self.layers)

    for k in range(num_layers-1):
      prev_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      prev_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

    for k in range(num_layers-1):
      update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      update_b[k+1] =np.zeros((self.layers[k + 1], 1))
    num_points_seen = 0
    for epoch in range(epochs):
      
      loss_per_point=0
      #initializing del_w and del_b
      
      for k in range(num_layers-1):
        del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
        del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
      

      for i in range(X_train.shape[1]):
        Y,H,A = self.forwardPropagate(self.X_train[:,i].reshape(784,1), self.W, self.b) 
        grad_w, grad_b = self.backPropagate(Y,H,A,self.Y_train[:,i].reshape(10,1))
        
        
        #add grad of weights and biases to get del_w and del_b
        for k in range(num_layers-1):
          del_w[k+1]=del_w[k+1] + grad_w[k+1]
          del_b[k+1]=del_b[k+1] + grad_b[k+1]

        loss_per_point= loss_per_point+ self.cross_entropy_loss(Y_train[:, i].reshape(10, 1), Y )+self.L2_loss(lamda)
      
      
        num_points_seen +=1
      
        if num_points_seen % batch_size == 0:
          #updating history in batch
          for k in range(num_layers-1):
            update_w[k+1]=gamma*prev_update_w[k+1]+eta*del_w[k+1] 
            update_b[k+1]=gamma*prev_update_b[k+1]+eta*del_b[k+1]

          for k in range(num_layers-1):
            self.W[k+1]=self.W[k+1] - learning_rate * update_w[k+1]
            self.b[k+1]=self.b[k+1] - learning_rate * update_b[k+1]

          

          prev_update_w = update_w
          prev_update_b = update_b
          
          for k in range(num_layers-1):
            del_w[k+1] = np.zeros((self.layers[k + 1],self.layers[k]))
            del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
      #loss    
      loss_train.append(loss_per_point/X_train.shape[1])

      #accuracy 
      Y_pred_final = self.predict(self.X_train,self.num_examples_train)
      acc_train.append(self.accuracy(Y_train, Y_pred_final, num_examples))
      acc_val.append(self.accuracy(self.Y_val, self.predict(self.X_val, self.num_examples_val), self.num_examples_val))

      print("epoch:",epoch,"training loss:",loss_train[epoch],"training accuracy",acc_train[epoch],"validation accuracy",acc_val[epoch])

      wandb.log({'epoch':epoch,'training_loss':loss_train[epoch],'training_accuracy':acc_train[epoch], 'validation_accuracy':acc_val[epoch] })


    return Y_pred_final,acc_val



  def nag(self,epochs,num_examples,batch_size,learning_rate,lamda):
    
    #gamma = min(1 - 2 ** (-1 - np.log((calls / 250.0) + 1, 2)), gamma)
    
    gamma=0.9
    eta=learning_rate
    loss_train = []
    acc_train = []
    acc_val = []
    del_w={}
    del_b={}
    prev_update_w={}
    prev_update_b={}
    update_w={}
    update_b={}
    num_layers=len(self.layers)
    
    X_train = self.X_train[:, :num_examples]
    Y_train = self.Y_train[:, :num_examples]        

    #initalizing prev and latest update
    for k in range(num_layers-1):
      prev_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      prev_update_b[k+1] =np.zeros((self.layers[k + 1], 1))
    
    for k in range(num_layers-1):
      update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      update_b[k+1] =np.zeros((self.layers[k + 1], 1))
    num_points_seen = 0
    
    #begin
    for epoch in range(epochs):
      
      loss_per_point=0
      #initializing del_w and del_b
      
      for k in range(num_layers-1):
        del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
        del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
      #doing lookahead
      for k in range(num_layers-1):
        update_w[k+1] = gamma*prev_update_w[k+1]
        update_b[k+1] =gamma*prev_update_b[k+1]

      for i in range(X_train.shape[1]):
        
        #look ahead w,b
        for k in range(num_layers-1):
          self.W[k+1]=self.W[k+1]-update_w[k+1]
          self.b[k+1]=self.b[k+1]-update_b[k+1]
          
        Y,H,A = self.forwardPropagate(self.X_train[:,i].reshape(784,1), self.W, self.b) 
        grad_w, grad_b = self.backPropagate(Y,H,A,self.Y_train[:,i].reshape(10,1))
        
        #add grad of weights and biases to get del_w and del_b
        for k in range(num_layers-1):
          del_w[k+1]=del_w[k+1] + grad_w[k+1]
          del_b[k+1]=del_b[k+1] + grad_b[k+1]
    
        loss_per_point= loss_per_point+ self.cross_entropy_loss(Y_train[:, i].reshape(10, 1), Y )+self.L2_loss(lamda)
      
      
        num_points_seen +=1
      
        if num_points_seen % batch_size == 0:
          
          #updating history in batch
          for k in range(num_layers-1):
            update_w[k+1]=gamma*prev_update_w[k+1]+eta*del_w[k+1] 
            update_b[k+1]=gamma*prev_update_b[k+1]+eta*del_b[k+1]

          for k in range(num_layers-1):
            self.W[k+1]=self.W[k+1] - eta * update_w[k+1]
            self.b[k+1]=self.b[k+1] - eta * update_b[k+1]

          

          prev_update_w = update_w
          prev_update_b = update_b
          for k in range(num_layers-1):
            del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
            del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
          
      #loss    
      loss_train.append(loss_per_point/X_train.shape[1])

      #accuracy 
      Y_pred_final = self.predict(self.X_train,self.num_examples_train)
      acc_train.append(self.accuracy(Y_train, Y_pred_final, num_examples))
      acc_val.append(self.accuracy(self.Y_val, self.predict(self.X_val, self.num_examples_val), self.num_examples_val))

      print("epoch:",epoch,"training loss:",loss_train[epoch],"training accuracy",acc_train[epoch],"validation accuracy",acc_val[epoch])

      wandb.log({'epoch':epoch,'training_loss':loss_train[epoch],'training_accuracy':acc_train[epoch], 'validation_accuracy':acc_val[epoch] })


    return Y_pred_final,acc_val

  def rmsprop(self,epochs,num_examples, batch_size, learning_rate, lamda):
    
    eta=learning_rate
    loss_train = []
    acc_train = []
    acc_val = []
    del_w={}
    del_b={}
    
    update_w={}
    update_b={}
    eps=10**(-8)
    beta=0.9

    num_layers=len(self.layers)
    X_train = self.X_train[:, :num_examples]
    Y_train = self.Y_train[:, :num_examples]


    #initialize update
    for k in range(num_layers-1):
      update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      update_b[k+1] =np.zeros((self.layers[k + 1], 1))
    
    num_points_seen = 0
    
    for epoch in range(epochs):
      
      loss_per_point=0
      #initializing del_w and del_b
      
      for k in range(num_layers-1):
        del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
        del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
      

      for i in range(X_train.shape[1]):
        Y,H,A = self.forwardPropagate(self.X_train[:,i].reshape(784,1), self.W, self.b) 
        grad_w, grad_b = self.backPropagate(Y,H,A,self.Y_train[:,i].reshape(10,1))
        
        #add grad of weights and biases to get del_w and del_b
        for k in range(num_layers-1):
          del_w[k+1]=del_w[k+1] + grad_w[k+1]
          del_b[k+1]=del_b[k+1] + grad_b[k+1]
    
        loss_per_point= loss_per_point+ self.cross_entropy_loss(Y_train[:, i].reshape(10, 1), Y )+self.L2_loss(lamda)
      
      
        num_points_seen +=1
      
        if num_points_seen % batch_size == 0:
          #updating history in batch
          for k in range(num_layers-1):
            update_w[k+1]=beta*update_w[k+1]+ (1-beta)*((del_w[k+1])**2) 
            update_b[k+1]=beta*update_b[k+1]+ (1-beta)*((del_b[k+1])**2)

          for k in range(num_layers-1):
            self.W[k+1]=self.W[k+1] - ((eta)/(np.sqrt(update_w[k+1]+eps)))*del_w[k+1]
            self.b[k+1]=self.b[k+1] - ((eta)/(np.sqrt(update_b[k+1]+eps)))*del_b[k+1]

        
          for k in range(num_layers-1):
            del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
            del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
          
              
      #loss    
      loss_train.append(loss_per_point/X_train.shape[1])

      #accuracy 
      Y_pred_final = self.predict(self.X_train,self.num_examples_train)
      acc_train.append(self.accuracy(Y_train, Y_pred_final, num_examples))
      acc_val.append(self.accuracy(self.Y_val, self.predict(self.X_val, self.num_examples_val), self.num_examples_val))

      print("epoch:",epoch,"training loss:",loss_train[epoch],"training accuracy",acc_train[epoch],"validation accuracy",acc_val[epoch])

      wandb.log({'epoch':epoch,'training_loss':loss_train[epoch],'training_accuracy':acc_train[epoch], 'validation_accuracy':acc_val[epoch] })

    return Y_pred_final,acc_val

  
  
  def adam(self, epochs,num_examples, batch_size, learning_rate, lamda):
        
    
    X_train = self.X_train[:, :num_examples]
    Y_train = self.Y_train[:, :num_examples]        
    num_layers=len(self.layers)
    

    eta=learning_rate
    loss_train = []
    acc_train = []
    acc_val = []
    del_w={}
    del_b={}
    
    m_update_w={}
    m_update_b={}

    m_bias_corr_update_w={}
    m_bias_corr_update_b={}
    
    v_update_w={}
    v_update_b={}

    v_bias_corr_update_w={}
    v_bias_corr_update_b={}
    
    eps=10**(-8)
    beta1=0.9
    beta2=0.95

    
    #initialize update
    for k in range(num_layers-1):
      m_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      m_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

      m_bias_corr_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      m_bias_corr_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

      v_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      v_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

      v_bias_corr_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      v_bias_corr_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

    num_points_seen = 0
    
    for epoch in range(epochs):
      
      loss_per_point=0
      #initializing del_w and del_b
      
      for k in range(num_layers-1):
        del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
        del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
      

      for i in range(X_train.shape[1]):
        Y,H,A = self.forwardPropagate(self.X_train[:,i].reshape(784,1), self.W, self.b) 
        grad_w, grad_b = self.backPropagate(Y,H,A,self.Y_train[:,i].reshape(10,1))
        
        #add grad of weights and biases to get del_w and del_b
        for k in range(num_layers-1):
          del_w[k+1]=del_w[k+1] + grad_w[k+1]
          del_b[k+1]=del_b[k+1] + grad_b[k+1]
    
        #loss_per_point= loss_per_point+ self.cross_entropy_loss(Y_train[:, i].reshape(10, 1), Y )+self.L2_loss(lamda)
        loss_per_point= loss_per_point+ self.mse_loss(Y_train[:, i].reshape(10, 1), Y )+self.L2_loss(lamda)
      
      
        num_points_seen +=1
      
        if num_points_seen % batch_size == 0:
          #updating history in batch
          for k in range(num_layers-1):
            
            m_update_w[k+1]=beta1*m_update_w[k+1]+ (1-beta1)*(del_w[k+1]) 
            m_update_b[k+1]=beta1*v_update_b[k+1]+ (1-beta1)*(del_b[k+1])
            
            v_update_w[k+1]=beta2*v_update_w[k+1]+ (1-beta2)*((del_w[k+1])**2) 
            v_update_b[k+1]=beta2*v_update_b[k+1]+ (1-beta2)*((del_b[k+1])**2)

            #beta**epoch+1 as epoch=0 it will be inf
            m_bias_corr_update_w[k+1]=m_update_w[k+1]/(1-((beta1)**(epoch+1)))
            m_bias_corr_update_b[k+1]=m_update_b[k+1]/(1-((beta1)**(epoch+1)))

            v_bias_corr_update_w[k+1]=v_update_w[k+1]/(1-((beta2)**(epoch+1)))
            v_bias_corr_update_b[k+1]=v_update_b[k+1]/(1-((beta2)**(epoch+1)))

          for k in range(num_layers-1):
            self.W[k+1]=self.W[k+1] - ((m_bias_corr_update_w[k+1])*(eta))/(np.sqrt(v_bias_corr_update_w[k+1]+eps))
            self.b[k+1]=self.b[k+1] - ((m_bias_corr_update_b[k+1])*(eta))/(np.sqrt(v_bias_corr_update_b[k+1]+eps))

        
          for k in range(num_layers-1):
            del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
            del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      

      #loss    
      loss_train.append(loss_per_point/X_train.shape[1])

      #accuracy 
      Y_pred_final = self.predict(self.X_train,self.num_examples_train)
      acc_train.append(self.accuracy(Y_train, Y_pred_final, num_examples))
      acc_val.append(self.accuracy(self.Y_val, self.predict(self.X_val, self.num_examples_val), self.num_examples_val))

      print("epoch:",epoch,"training loss:",loss_train[epoch],"training accuracy",acc_train[epoch],"validation accuracy",acc_val[epoch])

      wandb.log({'epoch':epoch,'training_loss':loss_train[epoch],'training_accuracy':acc_train[epoch], 'validation_accuracy':acc_val[epoch] })



    return Y_pred_final,acc_val


  def nadam(self, epochs,num_examples, batch_size, learning_rate, lamda ):
        
    
    X_train = self.X_train[:, :num_examples]
    Y_train = self.Y_train[:, :num_examples]        
    num_layers=len(self.layers)
    

    eta=learning_rate
    loss_train = []
    acc_train = []
    acc_val = []
    del_w={}
    del_b={}
    
    m_update_w={}
    m_update_b={}

    m_bias_corr_update_w={}
    m_bias_corr_update_b={}
    
    v_update_w={}
    v_update_b={}

    v_bias_corr_update_w={}
    v_bias_corr_update_b={}
    
    eps=10**(-8)
    beta1=0.9
    beta2=0.95

    
    #initialize update
    for k in range(num_layers-1):
      m_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      m_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

      m_bias_corr_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      m_bias_corr_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

      v_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      v_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

      v_bias_corr_update_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
      v_bias_corr_update_b[k+1] =np.zeros((self.layers[k + 1], 1))

    num_points_seen = 0
    
    for epoch in range(epochs):
      
      loss_per_point=0
      #initializing del_w and del_b
      
      for k in range(num_layers-1):
        del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
        del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      
      

      for i in range(X_train.shape[1]):
        Y,H,A = self.forwardPropagate(self.X_train[:,i].reshape(784,1), self.W, self.b) 
        grad_w, grad_b = self.backPropagate(Y,H,A,self.Y_train[:,i].reshape(10,1))
        
        #add grad of weights and biases to get del_w and del_b
        for k in range(num_layers-1):
          del_w[k+1]=del_w[k+1] + grad_w[k+1]
          del_b[k+1]=del_b[k+1] + grad_b[k+1]
    
        #loss_per_point= loss_per_point+ self.cross_entropy_loss(Y_train[:, i].reshape(10, 1), Y )+self.L2_loss(lamda)
        loss_per_point= loss_per_point+ self.mse_loss(Y_train[:, i].reshape(10, 1), Y )+self.L2_loss(lamda)
      
        num_points_seen +=1
      
        if num_points_seen % batch_size == 0:
          #updating history in batch
          for k in range(num_layers-1):
            
            m_update_w[k+1]=beta1*m_update_w[k+1]+ (1-beta1)*(del_w[k+1]) 
            m_update_b[k+1]=beta1*v_update_b[k+1]+ (1-beta1)*(del_b[k+1])
            
            v_update_w[k+1]=beta2*v_update_w[k+1]+ (1-beta2)*((del_w[k+1])**2) 
            v_update_b[k+1]=beta2*v_update_b[k+1]+ (1-beta2)*((del_b[k+1])**2)

            #beta**epoch+1 as epoch=0 it will be inf
            m_bias_corr_update_w[k+1]=m_update_w[k+1]/(1-((beta1)**(epoch+1)))
            m_bias_corr_update_b[k+1]=m_update_b[k+1]/(1-((beta1)**(epoch+1)))

            v_bias_corr_update_w[k+1]=v_update_w[k+1]/(1-((beta2)**(epoch+1)))
            v_bias_corr_update_b[k+1]=v_update_b[k+1]/(1-((beta2)**(epoch+1)))

            
          for k in range(num_layers-1):
            self.W[k+1]=self.W[k+1] - ((beta1*m_bias_corr_update_w[k+1]+((1-beta1)/(1-(beta1)**(epoch+1)))*del_w[k+1])*(eta))/((np.sqrt(v_bias_corr_update_w[k+1]+eps)))
            self.b[k+1]=self.b[k+1] - ((beta1*m_bias_corr_update_b[k+1]+((1-beta1)/(1-(beta1)**(epoch+1)))*del_b[k+1])*(eta)/(np.sqrt(v_bias_corr_update_b[k+1]+eps)))

        
          for k in range(num_layers-1):
            del_w[k+1] = np.zeros((self.layers[k + 1], self.layers[k]))
            del_b[k+1] =np.zeros((self.layers[k + 1], 1))
      

      #loss    
      loss_train.append(loss_per_point/X_train.shape[1])

      #accuracy 
      Y_pred_final = self.predict(self.X_train,self.num_examples_train)
      acc_train.append(self.accuracy(Y_train, Y_pred_final, num_examples))
      acc_val.append(self.accuracy(self.Y_val, self.predict(self.X_val, self.num_examples_val), self.num_examples_val))

      print("epoch:",epoch,"training loss:",loss_train[epoch],"training accuracy",acc_train[epoch],"validation accuracy",acc_val[epoch])

      wandb.log({'epoch':epoch,'training_loss':loss_train[epoch],'training_accuracy':acc_train[epoch], 'validation_accuracy':acc_val[epoch] })



    return Y_pred_final,acc_val





In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {"name":"Bayesian Sweep",
    'method':'bayes'
    }
metric_dict={"name":"validation_accuracy","goal":"maximize"}
sweep_config['metric'] = metric_dict

parameters_dict = {
    'optimizer': {
        'values': ['adam','nadam']
        },
    'batch_size': {
        'values': [16, 32, 64]
        },
    'max_epochs': {
          'values': [5, 10]
        },
    'initializer': {
            'values': ["random", "xavier"]
        },

    'num_hidden_layers': {
            'values': [3, 4, 5]
        },
    'num_hidden_neurons':{'values':[32,64,128]
                          },

    'learning_rate':{'values':[0.001,0.0001]
                     },

    'lamda':{'values':[0,0.5,0.0005]
             },
    
    'activation': {
            'values': ['tanh','sigmoid']
        }
        

    }
sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config,project='Mse_loss_assignment1')

Create sweep with ID: uhpmy0e2
Sweep URL: https://wandb.ai/dl22/Mse_loss_assignment1/sweeps/uhpmy0e2


In [ ]:
def train_nw():
  config_dict ={"max_epochs":10,
            "num_hidden_layers":3,
            "num_hidden_neurons":32,
            "lamda":0.0005,
            "learning_rate":0.001,
            "optimizer":"rmsprop",
            "batch_size":32,
            "activation":"tanh",
            "initializer":"random",
            "loss":"mse",
           }


  wandb.init(config = config_dict)

  wandb.run.name="hl_"+str(wandb.config.num_hidden_layers)+"_hn_"+str(wandb.config.num_hidden_neurons)+"_lr_"+str(wandb.config.learning_rate)+ "_opt_"+str(wandb.config.optimizer)+"_act_"+str(wandb.config.activation)+"_in_"+str(wandb.config.initializer)+"_bs_"+str(wandb.config.batch_size)
  
  nn =neural_network2(num_hidden_layers=wandb.config.num_hidden_layers,
        num_hidden_neurons=wandb.config.num_hidden_neurons,
        X_train=X_train,
        Y_train=Y_train,
        num_examples_train = num_examples_train,
        X_val = X_val,
        Y_val = Y_val,
        num_examples_val = num_examples_val,
        optimizer = wandb.config.optimizer,
        batch_size = wandb.config.batch_size,
        lamda = wandb.config.lamda,
        learning_rate = wandb.config.learning_rate,
        max_epochs = wandb.config.max_epochs,
        activation = wandb.config.activation,
        initializer = wandb.config.initializer,
        loss = wandb.config.loss
        )

  Y_pred_train,validation_accuracy = nn.optimizer(nn.max_epochs, nn.num_examples_train, nn.batch_size, nn.learning_rate,nn.lamda)

In [ ]:
wandb.agent(sweep_id,train_nw, count = 30)

wandb: Agent Starting Run: 9yyab3kf with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	initializer: xavier
wandb: 	lamda: 0.0005
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	num_hidden_layers: 5
wandb: 	num_hidden_neurons: 64
wandb: 	optimizer: adam
wandb: Currently logged in as: dl22 (use `wandb login --relogin` to force relogin)


epoch: 0 training loss: 0.13890053132143823 training accuracy 0.8351296296296297 validation accuracy 0.829
epoch: 1 training loss: 0.10710135497248852 training accuracy 0.8495370370370371 validation accuracy 0.8415
epoch: 2 training loss: 0.09737177758166375 training accuracy 0.8777592592592592 validation accuracy 0.8611666666666666
epoch: 3 training loss: 0.09157062689521855 training accuracy 0.8763518518518518 validation accuracy 0.864
epoch: 4 training loss: 0.08803368915964634 training accuracy 0.8817592592592592 validation accuracy 0.8598333333333333


epoch,▁▃▅▆█
training_accuracy,▁▃▇▇█
training_loss,█▄▂▁▁
validation_accuracy,▁▃▇█▇
epoch,4
training_accuracy,0.88176
training_loss,0.08803
validation_accuracy,0.85983


wandb: Agent Starting Run: 4leywg2g with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	initializer: xavier
wandb: 	lamda: 0.0005
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	num_hidden_layers: 5
wandb: 	num_hidden_neurons: 64
wandb: 	optimizer: nadam


epoch: 0 training loss: 0.3687196690162817 training accuracy 0.6370925925925925 validation accuracy 0.6365
epoch: 1 training loss: 0.22614578926499268 training accuracy 0.7174814814814815 validation accuracy 0.7111666666666666
epoch: 2 training loss: 0.1887874898855703 training accuracy 0.7703518518518518 validation accuracy 0.7656666666666667
epoch: 3 training loss: 0.1664253694723125 training accuracy 0.7885925925925926 validation accuracy 0.7831666666666667
epoch: 4 training loss: 0.15483502318930917 training accuracy 0.8003703703703704 validation accuracy 0.7918333333333333
epoch: 5 training loss: 0.1480106526227338 training accuracy 0.8086851851851852 validation accuracy 0.7978333333333333
epoch: 6 training loss: 0.1432600206030369 training accuracy 0.8170185185185185 validation accuracy 0.8028333333333333
epoch: 7 training loss: 0.1394344321850973 training accuracy 0.8255555555555556 validation accuracy 0.8123333333333334
epoch: 8 training loss: 0.1361096435639762 training accura

...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 0j2fvrz5 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	initializer: xavier
wandb: 	lamda: 0
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	num_hidden_layers: 5
wandb: 	num_hidden_neurons: 64
wandb: 	optimizer: nadam


epoch: 0 training loss: 0.1361204256620508 training accuracy 0.8626111111111111 validation accuracy 0.8521666666666666
epoch: 1 training loss: 0.09951125135044 training accuracy 0.8747962962962963 validation accuracy 0.8641666666666666
epoch: 2 training loss: 0.09171336031238778 training accuracy 0.8816111111111111 validation accuracy 0.8673333333333333
epoch: 3 training loss: 0.08703094617340436 training accuracy 0.885425925925926 validation accuracy 0.8703333333333333
epoch: 4 training loss: 0.08356517226289438 training accuracy 0.8883888888888889 validation accuracy 0.8705
epoch: 5 training loss: 0.08079256686760837 training accuracy 0.8917592592592593 validation accuracy 0.871
epoch: 6 training loss: 0.07847880613767282 training accuracy 0.8948703703703703 validation accuracy 0.8721666666666666
epoch: 7 training loss: 0.07648778091964482 training accuracy 0.8973888888888889 validation accuracy 0.8743333333333333
epoch: 8 training loss: 0.07474242189269699 training accuracy 0.899777

epoch,▁▂▃▃▄▅▆▆▇█
training_accuracy,▁▃▄▅▆▆▇▇██
training_loss,█▄▃▃▂▂▂▁▁▁
validation_accuracy,▁▄▅▆▆▆▇▇██
epoch,9
training_accuracy,0.90178
training_loss,0.07319
validation_accuracy,0.87717


wandb: Agent Starting Run: b975jghj with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	initializer: xavier
wandb: 	lamda: 0
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	num_hidden_layers: 5
wandb: 	num_hidden_neurons: 128
wandb: 	optimizer: nadam


epoch: 0 training loss: 0.12920621855840145 training accuracy 0.8609259259259259 validation accuracy 0.8463333333333334
epoch: 1 training loss: 0.09789024275537066 training accuracy 0.8697222222222222 validation accuracy 0.8528333333333333
epoch: 2 training loss: 0.09023735033176743 training accuracy 0.8763518518518518 validation accuracy 0.859
